In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
    "X": "something"
}
step_params={
    "X": "something_else"
}
substep_params={
    "param1":"None1",
    "param2":"None2"}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, get_user, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
   
    inputs =
    [
        { STEP_NAME: "step2", ENTITY_NAME: "entityD" },
        { STEP_NAME: "step2", ENTITY_NAME: "entityE" },
        { STEP_NAME: "step3", ENTITY_NAME: "entityF" }
    ],
    outputs = 
    [
        { ENTITY_NAME: "entityG" },
        { ENTITY_NAME: "entityH" }
    ],
    custom_outputs = 
    [
        { ENTITY_NAME: "custom_data", ENTITY_PATH: f"/data/home/{get_user()}/custom_output_data" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
#5 read inputs 
prev_step2_inputs = substep.inputs(step_name="step2")

dfD = spark.read.parquet(prev_step2_inputs.entityD).to_pandas_on_spark()
dfE = spark.read.parquet(prev_step2_inputs.entityE).to_pandas_on_spark()

prev_step3_inputs = substep.inputs(step_name="step3")

dfF = spark.read.parquet(prev_step3_inputs.entityF).to_pandas_on_spark()

In [ ]:
#6 make something to create artifacts

dfG = dfD.to_spark()
dfH = dfE.to_spark()
dfCustom = dfF.to_spark()

In [ ]:
#7 write outputs
outputs = substep.outputs()

dfG.write.parquet(outputs.entityG, mode='overwrite')
dfH.write.parquet(outputs.entityH, mode='overwrite')

custom_outputs = substep.custom_outputs()

dfCustom.write.parquet(custom_outputs.custom_data, mode='overwrite')

In [ ]:
#8 stop spark
SinaraSpark.stop_session()